<a href="https://colab.research.google.com/github/quintanaMathe/causal-analysis-netsi/blob/master/CrossLanguageEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross-Language Word Embeddings

We have mentioned, and will discuss in more detail this week, how we can reduce the dimensionality of word representations from their original vectors space to an embedding space on the order of a few hundred dimensions. Different modeling choices for word embeddings may be ultimately evaluated by the effectiveness of classifiers, parsers, and other inference models that use those embeddings.

In this assignment, however, we will consider another common method of evaluating word embeddings: by judging the usefulness of pairwise distances between words in the embedding space.

Follow along with the examples in this notebook, and implement the sections of code flagged with **TODO**.

In [11]:
import gensim
import numpy as np
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

We'll start by downloading a plain-text version of the Shakespeare plays we used for the first assignment.

In [12]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
lines = [s.split() for s in open('shakespeare_plays.txt')]

--2020-12-18 16:16:32--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/shakespeare_plays.txt
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4746840 (4.5M) [text/plain]
Saving to: ‘shakespeare_plays.txt’

shakespeare_plays.t 100%[===================>]   4.53M  19.5MB/s    in 0.2s    

2020-12-18 16:16:32 (19.5 MB/s) - ‘shakespeare_plays.txt’ saved [4746840/4746840]



Then, we'll estimate a simple word2vec model on the Shakespeare texts.

In [13]:
model = Word2Vec(lines)

Even with such a small training set size, you can perform some standard analogy tasks.

In [14]:
model.wv.most_similar(positive=['king','woman'], negative=['man'])

[('queen', 0.8097108602523804),
 ('prince', 0.7789651155471802),
 ('duke', 0.750512957572937),
 ('york', 0.733109712600708),
 ('clarence', 0.7240618467330933),
 ('warwick', 0.7200621366500854),
 ('princess', 0.7168160676956177),
 ('son', 0.7116094827651978),
 ('gloucester', 0.7084842920303345),
 ('widow', 0.6990600824356079)]

For the rest of this assignment, we will focus on finding words with similar embeddings, both within and across languages. For example, what words are similar to the name of the title character of *Othello*?

In [15]:
model.wv.most_similar(positive=['othello'])
#model.wv.most_similar(positive=['brutus'])

[('cassio', 0.9346123337745667),
 ('desdemona', 0.9329387545585632),
 ('emilia', 0.9260089993476868),
 ('ham', 0.9221329689025879),
 ('cleopatra', 0.9187358021736145),
 ('iago', 0.9144153594970703),
 ('troilus', 0.9024564623832703),
 ('celia', 0.897714376449585),
 ('pisanio', 0.8976635932922363),
 ('proteus', 0.8972246050834656)]

This search uses cosine similarity. In the default API, you should see the same similarity between the words `othello` and `desdemona` as in the search results above.

In [16]:
model.wv.similarity('othello', 'desdemona')

0.9329388

**TODO**: Your **first task**, therefore, is to implement your own cosine similarity function so that you can reuse it outside of the context of the gensim model object.

In [17]:
## TODO: Implement cosim
def cosim(v1, v2):
  ## return cosine similarity between v1 and v2
  return np.dot(v1,v2)/(np.linalg.norm(v2)*np.linalg.norm(v1))
  
## This should give a result similar to model.wv.similarity:
cosim(model.wv['othello'], model.wv['desdemona'])

0.9329388

## Evaluation

We could collect a lot of human judgments about how similar pairs of words, or pairs of Shakespearean characters, are. Then we could compare different word-embedding models by their ability to replicate these human judgments.

If we extend our ambition to multiple languages, however, we can use a word translation task to evaluate word embeddings.

We will use a subset of [Facebook AI's FastText cross-language embeddings](https://fasttext.cc/docs/en/aligned-vectors.html) for several languages. Your task will be to compare English both to French, and to *one more language* from the following set: Arabic, German, Portuguese, Russian, Spanish, Vietnamese, and Chinese.

In [19]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec

# TODO: uncomment at least one of these to work with another language
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ar.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.de.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.pt.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.ru.vec
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.es.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.vi.vec
# !wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.zh.vec

--2020-12-18 16:18:46--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.en.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67681172 (65M)
Saving to: ‘30k.en.vec’

30k.en.vec          100%[===================>]  64.54M  66.0MB/s    in 1.0s    

2020-12-18 16:18:47 (66.0 MB/s) - ‘30k.en.vec’ saved [67681172/67681172]

--2020-12-18 16:18:47--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/30k.fr.vec
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67802327 (65M)
Saving to: ‘30k.fr.vec’

30k.fr.vec          100%[===================>]  64.66M  72.0MB/s    in 0.9s    

2020-12-18 16:18:48 (72.0 MB/s) - ‘30k.fr.vec’ saved [67802327/67802327]

--2020-12-18 16:18:48--  http://www.ccs.neu.

We'll start by loading the word vectors from their textual file format to a dictionary mapping words to numpy arrays.

In [20]:
def vecref(s):
  (word, srec) = s.split(' ', 1)
  return (word, np.fromstring(srec, sep=' '))

def ftvectors(fname):
  return { k:v for (k, v) in [vecref(s) for s in open(fname)] if len(v) > 1} 

envec = ftvectors('30k.en.vec')
frvec = ftvectors('30k.fr.vec')

# TODO: load vectors for one more language, such as zhvec (Chinese)
# arvec = ftvectors('30k.ar.vec')
# devec = ftvectors('30k.de.vec')
# ptvec = ftvectors('30k.pt.vec')
# ruvec = ftvectors('30k.ru.vec')
esvec = ftvectors('30k.es.vec')
# vivec = ftvectors('30k.vi.vec')
# zhvec = ftvectors('30k.zh.vec')

**TODO**: Your next task is to write a simple function that takes a vector and a dictionary of vectors and finds the most similar item in the dictionary. For this assignment, a linear scan through the dictionary using your `cosim` function from above is acceptible.

In [21]:
## TODO: implement this search function
def mostSimilar(vec, vecDict):
  ## Use your cosim function from above
  similarity = 0
  mostSimilar = ''
  for word,v in vecDict.items():
    sim = cosim(v,vec)
    if sim > similarity:
      similarity = sim
      mostSimilar = word
  return (mostSimilar, similarity)

## some example searches
[mostSimilar(envec[e], frvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('informatique', 0.5023827767603765),
 ('allemagne', 0.593718413875964),
 ('matrice', 0.5088361302065517),
 ('physique', 0.4555543434796394),
 ('fermentation', 0.3504105196166514)]

In [22]:
[mostSimilar(envec[e], esvec) for e in ['computer', 'germany', 'matrix', 'physics', 'yeast']]

[('computador', 0.5013697495254124),
 ('alemania', 0.6352798713596078),
 ('matriz', 0.4784864671614966),
 ('física', 0.4784845095690361),
 ('levadura', 0.5114917452709493)]

Some matches make more sense than others. Note that `computer` most closely matches `informatique`, the French term for *computer science*. If you looked further down the list, you would see `ordinateur`, the term for *computer*. This is one weakness of a focus only on embeddings for word *types* independent of context.

To evalute cross-language embeddings more broadly, we'll look at a dataset of links between Wikipedia articles.

In [23]:
!wget http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
links = [s.split() for s in open('links.tab')]

--2020-12-18 16:28:08--  http://www.ccs.neu.edu/home/dasmith/courses/cs6120/links.tab
Resolving www.ccs.neu.edu (www.ccs.neu.edu)... 52.70.229.197
Connecting to www.ccs.neu.edu (www.ccs.neu.edu)|52.70.229.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1408915 (1.3M)
Saving to: ‘links.tab.1’

links.tab.1         100%[===================>]   1.34M  7.64MB/s    in 0.2s    

2020-12-18 16:28:09 (7.64 MB/s) - ‘links.tab.1’ saved [1408915/1408915]



This `links` variable consists of triples of `(English term, language, term in that language)`. For example, here is the link between English `academy` and French `académie`:

In [24]:
links[302]

['academy', 'fr', 'académie']

**TODO**: Evaluate the English and French embeddings by computing the proportion of English Wikipedia articles whose corresponding French article is also the closest word in embedding space. Skip English articles not covered by the word embedding dictionary. Since many articles, e.g., about named entities have the same title in English and French, compute the baseline accuracy achieved by simply echoing the English title as if it were French. Remember to iterate only over English Wikipedia articles, not the entire embedding dictionary.

In [25]:
en_fr_links = [l for l in links if l[1] == 'fr']

In [45]:
## TODO: Compute English-French Wikipedia retrieval accuracy.
correct_pred = 0 
baseline_pred = 0
total_pred = 0
for i,link in enumerate(en_fr_links):
  if i%100 == 0:
    print(i)
  if link[0] in envec.keys():
    total_pred += 1
    pred = mostSimilar(envec[link[0]], frvec)[0]
    if pred == link[2]:
      correct_pred += 1
    if link[0] == link[2]:
      baseline_pred += 1

accuracy = correct_pred/total_pred
baselineAccuracy = baseline_pred/total_pred

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800


In [46]:
accuracy

0.5359205593271862

In [47]:
baselineAccuracy

0.6742324450298915

**TODO**: Compute accuracy and baseline (identity function) acccuracy for Englsih and another language besides French. Although the baseline will be lower for languages not written in the Roman alphabet (i.e., Arabic or Chinese), there are still many articles in those languages with headwords written in Roman characters.

In [ ]:
## TODO: Compute English-X Wikipedia retrieval accuracy.


Further evaluation, if you are interested, could involve looking at the $k$ nearest neighbors of each English term to compute "recall at 10" or "mean reciprocal rank at 10".